In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy.lang.en import English
import pandas as pd
import random
random.seed(2022)

In [ ]:
model = "en_core_web_trf"

In [ ]:
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

In [ ]:
df = pd.read_csv("../data/csvfiles/scraped_web/beta_announcements.csv")
df.head()

In [ ]:
titles = list(df['title'])

random.shuffle(titles)
ts = random.sample(titles, 10)
for t in ts:
    print(t)

In [ ]:
train_strs = {ts[0] : {"Beta" : "COMP", "Amazon's Climate Fund" : "PARTNER", "Fidelity" : "PARTNER"},
              ts[1] : {},
             ts[2] : {"Beta" : "COMP"},
             ts[3] : {"BETA Technologies":"COMP", "ALIA":"TECH"},
             ts[4] : {"Beta":"COMP", "Ava":"TECH"},
             ts[5] : {},
             ts[6] : {"Beta":"COMP", "Alia":"TECH"},
             ts[7] : {"Beta Technologies":"COMP"},
             ts[8] : {"Beta":"COMP"},
             ts[9] : {"Beta":"COMP", "Alia":"TECH"}}

In [ ]:
def patt_finder(substr, string):
    start_idx = string.index(substr)
    end_idx = start_idx + len(substr)
    return start_idx, end_idx

train_data = []
for i, sentence in enumerate(train_strs.keys()):
    assert(len(train_data)==i)
    num_ents = len(train_strs[sentence].keys())
    train_data.append([sentence, [None]*num_ents])
    for j in range(num_ents):
        sent_dict = train_strs[sentence]
        start_idx, end_idx = patt_finder(list(sent_dict.keys())[j], sentence)
        ent_str = list(sent_dict.values())[j]
        train_data[i][1][j] = (start_idx, end_idx, ent_str)

In [ ]:
train_data

In [ ]:
temp = [[lst[0], {"entities" : lst[1]}] for lst in train_data]
TRAINING_DATA = [tuple(dp) for dp in temp]
TRAINING_DATA

In [ ]:
# # Loop for 10 iterations
# for i in range(10):
#     # Shuffle the training data
#     random.shuffle(TRAINING_DATA)
#     # Create batches and iterate over them
#     for batch in spacy.util.minibatch(TRAINING_DATA, size=3):
#         # Split the batch in texts and annotations
#         texts = [text for text, annotation in batch]
#         annotations = [annotation for text, annotation in batch]
#         # Update the model
#         nlp.update(texts, annotations)
#         # Save the model
#         nlp.to_disk(path_to_model)